In [ ]:
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install tqdm
!pip install scipy
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/beomso0/ezSASRec.git

Cloning into 'ezSASRec'...
remote: Enumerating objects: 1078, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1078 (delta 47), reused 59 (delta 24), pack-reused 979
Receiving objects: 100% (1078/1078), 43.18 MiB | 20.37 MiB/s, done.
Resolving deltas: 100% (655/655), done.
Error downloading object: example/sasrec_data.txt (b418b8c): Smudge error: Error downloading example/sasrec_data.txt (b418b8c03bc84bc009cd7ac93e9753fdd82f57fa3f3f3a1f535b40d812fdd921): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/ezSASRec/.git/lfs/logs/20230504T171311.580983494.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: example/sasrec_data.txt: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD 

In [ ]:
!pwd
import pandas as pd 
import pickle
from ezSASRec.sasrec.util import filter_k_core, SASRecDataSet, load_model
from ezSASRec.sasrec.model import SASREC
from ezSASRec.sasrec.sampler import WarpSampler
import multiprocessing

/content


In [ ]:
df = pd.read_csv('ratings_small.csv')
print(df.head())
df = df.rename({'userId':'userID','movieId':'itemID','timestamp':'time'},axis=1)\
       .sort_values(by=['userID','time'])\
       .drop(['rating','time'],axis=1)\
       .reset_index(drop=True)

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [ ]:
df.head()

,userID,itemID
0,1,2294
1,1,2455
2,1,3671
3,1,1339
4,1,1343


In [ ]:
# filter data
# every user and item will appear more than 6 times in filtered_df

filtered_df = filter_k_core(df, 7)

Original: 671 users and 9066 items
Final: 671 users and 2794 items


In [ ]:
S# make maps (encoder)

user_set, item_set = set(filtered_df['userID'].unique()), set(filtered_df['itemID'].unique())
user_map = dict()
item_map = dict()
for u, user in enumerate(user_set):
    user_map[user] = u+1
for i, item in enumerate(item_set):
    item_map[item] = i+1

maps = (user_map, item_map)   

In [ ]:
# Encode filtered_df

filtered_df["userID"] = filtered_df["userID"].apply(lambda x: user_map[x])
filtered_df["itemID"] = filtered_df["itemID"].apply(lambda x: item_map[x])

In [ ]:
# save data and maps

# save sasrec data    
filtered_df.to_csv('sasrec_data.txt', sep="\t", header=False, index=False)

# save maps
with open('maps.pkl','wb') as f:
    pickle.dump(maps, f)

In [ ]:
# load data

data = SASRecDataSet('sasrec_data.txt')
data.split() # train, val, test split
              # the last interactions of each user is used for test
              # the last but one will be used for validation
              # others will be used for train

In [ ]:
# make model and warmsampler for batch training

max_len = 100
hidden_units = 128
batch_size = 2048

model = SASREC(
    item_num=data.itemnum,
    seq_max_len=max_len,
    num_blocks=2,
    embedding_dim=hidden_units,
    attention_dim=hidden_units,
    attention_num_heads=2,
    dropout_rate=0.2,
    conv_dims = [hidden_units, hidden_units],
    l2_reg=0.00001
)

sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, batch_size=batch_size, maxlen=max_len, n_workers=multiprocessing.cpu_count())

In [ ]:
# train model

model.train(
          data,
          sampler,
          num_epochs=3, 
          batch_size=batch_size, 
          lr=0.001, 
          val_epoch=1,
          val_target_user_n=1000, 
          target_item_n=-1,
          auto_save=True,
          path = path,
          exp_name='exp_example',
        )

epoch 1 / 3 -----------------------------


Evaluating...


epoch: 1, test (NDCG@10: 0.0005308601894307335, HR@10: 0.0014903129657228018)
best score model updated and saved
epoch 2 / 3 -----------------------------


Evaluating...


epoch: 2, test (NDCG@10: 0.0005308601894307335, HR@10: 0.0014903129657228018)
epoch 3 / 3 -----------------------------


Evaluating...


epoch: 3, test (NDCG@10: 0.0005308601894307335, HR@10: 0.0014903129657228018)


In [ ]:
# load trained model

model = load_model(path,'exp_example')

In [ ]:
# get user-item score

# make inv_user_map
inv_user_map = {v: k for k, v in user_map.items()}

# sample target_user
model.sample_val_users(data, 100)
encoded_users = model.val_users

# get scores
score = model.get_user_item_score(data,
                          [inv_user_map[u] for u in encoded_users], # user_list containing raw(not-encoded) userID 
                          [1,2,3], # item_list containing raw(not-encoded) itemID
                          user_map,
                          item_map,   
                          batch_size=10
                        )

10


100%|██████████| 10/10 [00:01<00:00,  9.65batch/s]


In [ ]:
score.head()

,user_id,1,2,3
0,2,0.460212,0.071222,0.129509
1,5,0.853707,0.233329,-0.392201
2,11,0.475220,0.374038,0.198571
3,12,0.767757,0.623964,0.254221
4,22,0.526562,-0.058280,-0.274096


In [ ]:
# get top N recommendation 

reco = model.recommend_item(data,
                            user_map,
                            [inv_user_map[u] for u in encoded_users],
                            is_test=True,
                            top_n=5)

100%|██████████| 100/100 [00:06<00:00, 14.93it/s]


In [ ]:
reco

{422: [(2003, 1.2224007),
  (1665, 1.1951469),
  (247, 1.1835097),
  (81, 1.1776268),
  (2019, 1.0858551)],
 260: [(2322, 1.088439),
  (2416, 0.95190465),
  (18, 0.9348159),
  (2213, 0.93174714),
  (214, 0.9065238)],
 406: [(2238, 1.3845417),
  (80, 1.0888664),
  (1053, 0.9918094),
  (675, 0.9756071),
  (1523, 0.9511444)],
 495: [(2511, 0.9650674),
  (5, 0.92675936),
  (1478, 0.90505385),
  (1583, 0.89823633),
  (913, 0.86421263)],
 312: [(966, 1.256087),
  (235, 1.1908715),
  (1689, 1.0916169),
  (1450, 1.000538),
  (723, 0.96569514)],
 421: [(74, 1.409432),
  (37, 1.1680356),
  (562, 1.1363487),
  (1708, 1.0174863),
  (1478, 0.9949125)],
 448: [(74, 1.4300488),
  (562, 1.3143786),
  (1478, 1.1511943),
  (304, 1.0249877),
  (1145, 1.0205306)],
 569: [(22, 1.104713),
  (1157, 1.054084),
  (2213, 0.9849154),
  (208, 0.9058619),
  (1313, 0.8798231)],
 467: [(692, 1.0833101),
  (268, 1.0286359),
  (997, 1.0165539),
  (1115, 1.0158104),
  (653, 1.0023532)],
 517: [(2704, 1.1338469),
  (743